In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import sys
sys.path.append("/content/drive/MyDrive/optiver-trading-at-the-close (1)/")

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization,Input,Dropout,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

stock_num = 200
date_num = 481
second_num = 55
training_num = 6000
epoch = 10
batch_size = 32

df = pd.read_csv("/content/drive/MyDrive/optiver-trading-at-the-close (1)/train.csv")
df = df[["stock_id", "date_id", "seconds_in_bucket", "target"]]
df.dropna(subset=["target"], inplace=True)
print(df)

         stock_id  date_id  seconds_in_bucket    target
0               0        0                  0 -3.029704
1               1        0                  0 -5.519986
2               2        0                  0 -8.389950
3               3        0                  0 -4.010200
4               4        0                  0 -7.349849
...           ...      ...                ...       ...
5237975       195      480                540  2.310276
5237976       196      480                540 -8.220077
5237977       197      480                540  1.169443
5237978       198      480                540 -1.540184
5237979       199      480                540 -6.530285

[5237892 rows x 4 columns]


In [4]:
def generate_features(df):

    df_pivoted = df.pivot(columns='stock_id', index=['date_id', 'seconds_in_bucket'], values='target')
    return df_pivoted

df_pivoted = generate_features(df)
df_pivoted=df_pivoted.fillna(0)
df_pivoted

stock_id                        0          1         2         3         4    \
date_id seconds_in_bucket                                                      
0       0                 -3.029704  -5.519986 -8.389950 -4.010200 -7.349849   
        10                 0.389814  -1.620054 -7.460117 -1.040101 -6.719828   
        20                 4.220009  -6.459951 -6.380081 -2.030134 -0.690222   
        30                 5.450249  -5.149841 -5.819797  1.130104 -2.589822   
        40                 3.169775  -6.750226 -0.280142 -2.350211 -4.900098   
...                             ...        ...       ...       ...       ...   
480     500                3.999472   2.850294 -4.450083  1.720190  1.939535   
        510                3.190041   3.169775 -5.559921  4.669428  2.139807   
        520               -0.169873   5.689859 -5.310178  5.639792  1.660585   
        530                3.110170  10.650158 -5.239844  5.229711  0.300407   
        540                0.760555  15.859604 -1.549721  3.160238  1.009703   

stock_id                        5         6          7         8         9    \
date_id seconds_in_bucket                                                      
0       0                  6.779432 -2.499819  -1.959801 -5.970001  7.970333   
        10                 7.710457 -4.280209  -8.010268 -0.780225 -0.640154   
        20                 1.009703  2.930164 -12.069941 -4.119873 -3.250241   
        30                -3.089905  0.560284  -6.909967 -1.090169 -6.759763   
        40                -2.070069  2.199411  -1.209974 -3.100038 -8.199811   
...                             ...       ...        ...       ...       ...   
480     500               -4.569888 -0.630021  -1.729727 -0.680089  3.540516   
        510                1.679659 -0.360012  -3.240108  2.609491  1.679659   
        520                2.110004 -1.000166  -4.529953  5.409718  0.350475   
        530               -2.030134 -1.180172  -1.310110  3.240108  0.220537   
        540                2.340078  1.410246  -1.260042  3.679991 -1.609922   

stock_id                   ...       190        191        192       193  \
date_id seconds_in_bucket  ...                                             
0       0                  ...  6.370544  11.940002 -11.529922 -6.489754   
        10                 ...  4.210472   2.599955 -15.599728 -1.749992   
        20                 ...  5.379915  -5.970001 -12.710094  2.160072   
        30                 ... -3.269911   1.300573 -16.660095  4.090071   
        40                 ... -2.009869  -0.969768  -9.229779  5.700588   
...                        ...       ...        ...        ...       ...   
480     500                ... -4.730225  -7.420182   2.959967 -2.359748   
        510                ... -3.340244  -3.479719   1.419783 -1.069903   
        520                ... -1.549721  -1.289845  -0.140071 -1.929998   
        530                ... -1.440048  -0.299811  -2.120137 -1.350045   
        540                ... -0.470281   0.799894  -1.729727 -3.880262   

stock_id                         194       195       196       197       198  \
date_id seconds_in_bucket                                                      
0       0                   3.999472 -0.690222 -0.810027 -8.440018 -0.510216   
        10                 -9.030104 -1.320243 -3.259778 -4.410148 -1.419783   
        20                 -9.999871  5.890131 -0.029802 -6.819964  2.599955   
        30                 -9.340048  7.020235  1.549721  1.690388  3.010035   
        40                -11.489987  4.500151 -1.350045 -1.929998  4.669428   
...                              ...       ...       ...       ...       ...   
480     500                -0.650287  3.190041 -1.659989 -0.690222 -2.700090   
        510                 2.319813  3.770590 -3.039837 -3.299713 -2.070069   
        520                 4.609823  1.449585 -6.319881 -1.999736 -2.239943   
        530                 3.110170  4.019737 -7

In [5]:
def build_model(dropout=0.5):
    model = Sequential()
    model.add(Input(shape=(55, 200)))
    model.add(Bidirectional(LSTM(200, activation="tanh", return_sequences=False)))
    model.add(Dropout(dropout))
    model.add(Dense(200))
    model.compile(loss='mae',
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
    return model

In [6]:
def windowed_dataset(dataset, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices(dataset)
    dataset = dataset.window(55 + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(55 + 1))
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    if shuffle:
      dataset = dataset.shuffle(10000)
    dataset = dataset.batch(64).prefetch(1)
    return dataset

In [10]:
split_index = int(len(df_pivoted) * 0.8)
train_df = df_pivoted.iloc[:split_index]
valid_df = df_pivoted.iloc[split_index:]
scaler=StandardScaler()
train_df = scaler.fit_transform(train_df)
valid_df = scaler.transform(valid_df)
train_dataset = windowed_dataset(train_df)
valid_dataset = windowed_dataset(valid_df, shuffle=False)
model = build_model()
history = model.fit(train_dataset,validation_data=valid_dataset,epochs=120,batch_size=64,verbose=True)


y_pred = model.predict(valid_dataset)
y_pred = scaler.inverse_transform(y_pred)
y_true = valid_df[55:]
mae = mean_absolute_error(y_true, y_pred)
print(f"MAE score: {mae}")

    ## Plots ##
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 200), dtype=tf.float64, name=None), TensorSpec(shape=(None, 200), dtype=tf.float64, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 200), dtype=tf.float64, name=None), TensorSpec(shape=(None, 200), dtype=tf.float64, name=None))>
Epoch 1/120
330/330 [==============================] - 12s 21ms/step - loss: 0.6801 - val_loss: 0.5942
Epoch 2/120
173/330 [==============>...............] - ETA: 2s - loss: 0.6185

KeyboardInterrupt: ignored

In [ ]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))
    sample_prediction['target'] = 0
    env.predict(sample_prediction)
    counter += 1